# Does My Design Meet Requirements?

**The old way:** Three engineers — requirements, architecture, CAD — meet for weeks to manually check that everything lines up.

**The new way:** Pull data from Istari, run automated checks, get a pass/fail report in seconds.

This notebook:
1. Extracts requirements and architecture from a SysML model (via SysGit)
2. Loads CAD results from an nTop wing run (aerodeck metrics)
3. Runs compliance checks — does the CAD meet the requirements?
4. Finds a failure, updates the requirement, and re-checks

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Milestone 1: Load everything from Istari
#
# Two models, one system:
#   - SysML model → extract to get requirements JSON + parts JSON
#   - nTop model  → load aerodeck metrics from latest run

SYSML_MODEL_ID = "22bb5f2d-b902-4418-a75c-9e1ae1504c53"  # Group3 UAS Requirements
NTOP_MODEL_ID = "063a1593-6fad-4dd9-b963-f0c8bf4119a1"   # Group3-UAS-Wing-v8

sysml_model = client.get_model(SYSML_MODEL_ID)
ntop_model = client.get_model(NTOP_MODEL_ID)

print(f"SysML model: {sysml_model.display_name or 'Group3 UAS Requirements'}")
print(f"  Artifacts: {len(sysml_model.artifacts)}")
print(f"\nnTop model: {ntop_model.display_name or 'Group3-UAS-Wing-v8'}")
print(f"  Artifacts: {len(ntop_model.artifacts)}")

# Load the extracted requirements and parts
reqs_data = parts_data = metrics_data = None

for a in sysml_model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_data = json.loads(a.read_text())
    elif rev and "parts" in rev.name and rev.name.endswith(".json"):
        parts_data = json.loads(a.read_text())

for a in reversed(ntop_model.artifacts):
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "aerodeck_metrics" in rev.name:
        metrics_data = json.loads(a.read_text())
        break

print(f"\nLoaded: {len(reqs_data)} requirements, {len(parts_data)} parts, aerodeck metrics")
print("\n--- Everything in one place. Ready to check. ---")

In [ ]:
# Quick look: what are the targets and actuals?
from IPython.display import HTML, display

range_info = metrics_data["range_mission"]
mass_info = metrics_data["mass_properties"]

html = """
<table style="border-collapse: collapse; font-size: 14px;">
<thead>
    <tr style="background: #f1f5f9;">
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Metric</th>
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Requirement</th>
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual (CAD)</th>
    </tr>
</thead>
<tbody>
    <tr><td style="padding: 6px 16px;">Range</td><td>\u2265 1,500 nm</td><td><b>""" + f"{range_info['range_nm']:,.0f} nm" + """</b></td></tr>
    <tr><td style="padding: 6px 16px;">Structure Weight</td><td>\u2264 275 lb</td><td><b>""" + f"{mass_info['empty_weight_lbm']} lb" + """</b></td></tr>
    <tr><td style="padding: 6px 16px;">Cruise Speed</td><td>\u2265 100 kts</td><td><b>""" + f"{range_info['cruise_speed_kts']} kts" + """</b></td></tr>
</tbody>
</table>"""
display(HTML(html))

In [ ]:
# Milestone 2: Run compliance checks
from compliance_checks import run_all_checks, format_report

results = run_all_checks(reqs_data, parts_data, metrics_data)
print(format_report(results))

# Show as HTML table
rows = ""
for r in results:
    color = "#059669" if r["status"] == "PASS" else "#dc2626"
    icon = "\u2705" if r["status"] == "PASS" else "\u274c"
    margin = f"+{r['margin']}%" if r["margin"] >= 0 else f"{r['margin']}%"
    rows += f'<tr><td style="padding: 6px 12px;">{r["check"]}</td>'
    rows += f'<td style="padding: 6px 12px;">{r["target"]} {r["unit"]}</td>'
    rows += f'<td style="padding: 6px 12px;">{r["actual"]} {r["unit"]}</td>'
    rows += f'<td style="padding: 6px 12px; color: {color}; font-weight: bold;">{icon} {r["status"]} ({margin})</td></tr>\n'

html = f"""
<table style="border-collapse: collapse; font-size: 14px; margin-top: 12px;">
<thead>
    <tr style="background: #f1f5f9;">
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Check</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Target</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Result</th>
    </tr>
</thead>
<tbody>{rows}</tbody>
</table>"""
display(HTML(html))

In [ ]:
# Milestone 3: Weight fails — update the requirement and re-check
#
# The wing weighs 321.8 lb but the requirement says 275 lb.
# The team agrees to relax the weight budget to 325 lb.
# Update the SysML, re-upload, re-extract, re-check.

import tempfile

print("Structure Weight: 321.8 lb vs 275 lb requirement --> FAIL")
print("Team decision: relax weight budget to 325 lb\n")

# Download current SysML, update the requirement
sysml_text = sysml_model.read_text()
updated_text = sysml_text.replace(
    'attribute maxValue : Real = 275.0;',
    'attribute maxValue : Real = 325.0;'
).replace(
    'shall not exceed 275 lb',
    'shall not exceed 325 lb'
)

# Verify the change
changes = 0
for old, new in zip(sysml_text.splitlines(), updated_text.splitlines()):
    if old != new:
        changes += 1
        print(f"  - {old.strip()}")
        print(f"  + {new.strip()}")
        print()
print(f"{changes} lines changed")

# Write to temp file and upload as new revision
with tempfile.NamedTemporaryFile(mode="w", suffix=".sysml", delete=False) as tmp:
    tmp.write(updated_text)
    tmp_path = Path(tmp.name)

client.update_model(model_id=SYSML_MODEL_ID, path=tmp_path)
tmp_path.unlink()  # clean up temp file
print("Uploaded new revision to Istari")

In [ ]:
# Re-extract requirements and re-run checks
from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Re-extracting requirements...")
job = client.add_job(
    model_id=SYSML_MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    # Reload updated requirements
    sysml_model = client.get_model(SYSML_MODEL_ID)
    for a in sysml_model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        if rev and "requirements" in rev.name and rev.name.endswith(".json"):
            reqs_data = json.loads(a.read_text())
            break

    # Re-run checks with updated requirements
    print("\n--- Updated compliance report ---\n")
    results = run_all_checks(reqs_data, parts_data, metrics_data)
    print(format_report(results))

    # Show updated HTML table
    rows = ""
    for r in results:
        color = "#059669" if r["status"] == "PASS" else "#dc2626"
        icon = "\u2705" if r["status"] == "PASS" else "\u274c"
        margin = f"+{r['margin']}%" if r["margin"] >= 0 else f"{r['margin']}%"
        rows += f'<tr><td style="padding: 6px 12px;">{r["check"]}</td>'
        rows += f'<td style="padding: 6px 12px;">{r["target"]} {r["unit"]}</td>'
        rows += f'<td style="padding: 6px 12px;">{r["actual"]} {r["unit"]}</td>'
        rows += f'<td style="padding: 6px 12px; color: {color}; font-weight: bold;">{icon} {r["status"]} ({margin})</td></tr>\n'

    html = f"""
    <table style="border-collapse: collapse; font-size: 14px; margin-top: 12px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Check</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Target</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Result</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>"""
    display(HTML(html))
else:
    print("Extraction failed!")

## Summary

**What we did:**
1. Pulled requirements (SysML) and CAD results (nTop aerodeck) from Istari
2. Ran automated compliance checks — found that weight **fails** (321.8 lb vs 275 lb)
3. Updated the requirement (275 → 325 lb), re-extracted, re-checked — **all green**

**Old way:** Weeks of meetings between requirements, architecture, and CAD engineers to manually check alignment.

**New way:** Everyone works in their own tool. Istari stores everything. Automated checks run in seconds. When something changes, re-check immediately.

### What's Next?

- **Add more checks** — extend `compliance_checks.py` with your own project-specific rules
- **Run in CI** — trigger checks automatically when any model is updated in Istari
- **Compare across runs** — check compliance for different nTop parameter sets to find the best design